### 先选几张图片显示看看

In [ ]:
import numpy as np
from matplotlib  import pyplot as plt
import matplotlib.image as mping
import cv2


In [ ]:
ids= ['1f1cc6b3a4','5b7c160d0d','6c40978ddf','7dfdf6eeb8','7e5a6e5013']
plt.figure(figsize=(20,10))
for j, img_name in enumerate(ids):
    q = j+1
    img = cv2.imread('./saltdata/train/images/' + img_name + '.png')
    img_mask = cv2.imread('./saltdata/train/masks/' + '5b7c160d0d' + '.png')
    
    plt.subplot(1,2*(1+len(ids)),q*2-1)
    plt.imshow(img)
    plt.subplot(1,2*(1+len(ids)),q*2)
    plt.imshow(img_mask)
plt.show()
#图片的像素值都比较大，所以表现为黑色
print(np.int32(np.bools(img_mask[0:100,0:100,0])))



In [ ]:
image_path = "./data/train/images/5b7c160d0d.png"
image = np.array(cv2.imread(image_path),dtype=np.float32)
mask = np.array(cv2.imread('./data/train/masks/5b7c160d0d.png'))
#print(image[:,:,0])
plt.imshow(image[:,:,0],cmap='gray')
plt.figure()
plt.imshow(mask)
print((image[:,:,0]==image[:,:,1]))
plt.figure()
plt.imshow(image)
plt.show()

### 将图片读入

In [ ]:
import csv
import pandas as pd
def load_img(path):
    return cv2.imread(path)
def get_ids(csv_path):
    #list_id中保存的是训练图片的名称，但是没有后缀
    df = pd.read_csv(csv_path)
    list_id = []
    for i,item in df.iterrows():
        list_id.append(item[0]+".png")
    return list_id

In [ ]:
import sys
import os
from tqdm import tqdm_notebook
import numpy as np
path_train = "./saltdata/train/"
path_test = "./saltdata/test/"
im_height, im_width, im_chan = 128, 128, 1
train_csv_path = "./saltdata/train.csv"
test_csv_path = "./saltdata/sample_submission.csv"
train_ids = get_ids(train_csv_path)
test_ids = get_ids(test_csv_path)
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), im_height, im_width, im_chan), dtype=np.float32)
##此处的存为np.uint8值得商榷
##需要做输入的标准化操作
Y_train = np.zeros((len(train_ids), im_height, im_width, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(train_ids), total=len(train_ids)):
    path = path_train
    img = load_img(path + '/images/' + id_)
    x = cv2.resize(img,(128,128))#cv2.resize只能处理三个通道的图片
    #x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)#为什么只取一个通道投入网络训练？
    #此处的resize是插值的方式，还是其他形式？
    #可对比resize前后的图片
    X_train[n] = np.expand_dims(np.array(x)[:,:,1],axis=-1)
    mask = np.array(load_img(path + '/masks/' + id_))
    mask = cv2.resize(mask, (128, 128))
    Y_train[n] = np.expand_dims(np.array(mask[:,:,0]),axis = -1)

print('Done!')

In [ ]:
# 检查下训练的数据是否读入正确
import random
ix = random.randint(0, len(train_ids))
plt.imshow(np.dstack((X_train[ix],X_train[ix],X_train[ix])))# 此处是对单通道的显示
plt.show()
tmp = np.squeeze(Y_train[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.show()

In [ ]:
# Define IoU metric
from keras import backend as K
from keras.layers.merge import concatenate
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D,Lambda,Conv2DTranspose
from keras.models import Model
import tensorflow as tf
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [ ]:
# Build U-Net model
inputs = Input((im_height, im_width, im_chan))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (s)
c1 = Conv2D(8, (3, 3), activation='relu', padding='same') (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (p1)
c2 = Conv2D(16, (3, 3), activation='relu', padding='same') (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (p2)
c3 = Conv2D(32, (3, 3), activation='relu', padding='same') (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (p3)
c4 = Conv2D(64, (3, 3), activation='relu', padding='same') (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (p4)
c5 = Conv2D(128, (3, 3), activation='relu', padding='same') (c5)

u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (u6)
c6 = Conv2D(64, (3, 3), activation='relu', padding='same') (c6)

u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (u7)
c7 = Conv2D(32, (3, 3), activation='relu', padding='same') (c7)

u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (u8)
c8 = Conv2D(16, (3, 3), activation='relu', padding='same') (c8)

u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])
model.summary()

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint
earlystopper = EarlyStopping(patience=5, verbose=1)
checkpointer = ModelCheckpoint('tgs-salt_model.{epoch:03d}.hdf5', verbose=1, save_best_only=True)

results = model.fit(X_train, Y_train, validation_split=0.1, batch_size=8, epochs=30)

In [ ]:
model.load_weights("model-tag-salt-1.h5")
ini_epoch = 0
results = model.fit(X_train, Y_train, initial_epoch=ini_epoch, validation_split=0.1, batch_size=8, epochs=30, 
                    callbacks=[earlystopper, checkpointer])

### resize测试图片

In [ ]:
# Get and resize test images
X_test = np.zeros((len(test_ids), im_height, im_width, im_chan), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm_notebook(enumerate(test_ids), total=len(test_ids)):
    path = path_test
    img = load_img(path + '/images/' + id_)
    x = np.array(img)[:,:,1]
    sizes_test.append([x.shape[0], x.shape[1]])
    #x = resize(x, (128, 128, 1), mode='constant', preserve_range=True)
    x = cv2.resize(x,(128,128))
    X_test[n] = np.expand_dims(np.array(x),axis=-1)

print('Done!')

### 在训练集，验证集，测试集上做预测

In [ ]:
# Predict on train, val and test
from keras.models import load_model
model = load_model('model-tgs-salt-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# 设定阈值，得到预测的结果
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

### 将得到的测试集预测图片进行反采样，从【128，,128】回到【101,101】大小

In [ ]:
# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(cv2.resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1])))

### 确认是否从采样会原来大小

In [ ]:
preds_test_upsampled[0].shape

### 随便从训练集中选张样本，对比其mask 和prediction

In [ ]:
# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
plt.imshow(np.dstack((X_train[ix],X_train[ix],X_train[ix])))
plt.show()
tmp = np.squeeze(Y_train[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.show()
tmp = np.squeeze(preds_train_t[ix]).astype(np.float32)
plt.imshow(np.dstack((tmp,tmp,tmp)))
plt.show()

## 将mask转换成比赛需要提交的格式

In [ ]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

#pred_dict = {fn[:-4]:RLenc(np.round(preds_test_upsampled[i])) for i,fn in tqdm_notebook(enumerate(test_ids))}

In [ ]:
raw_labels = np.zeros((4000,101,101),dtype="np.uint8")
for i, fn in enumerate(train_ids):
    
    raw_labels[i] = cv2.resize(np.dstack((Y_train[i],Y_train[i],Y_train[i])),(101,101))[:,:,0]

In [ ]:
pred_dict = {fn[:-4]:RLenc(np.round(cv2.resize(np.squeeze(Y_train[i]).astype(np.uint8),(101,101)))) for i,fn in tqdm_notebook(enumerate(train_ids))}

### 写入csv文件

In [ ]:
sub = pd.DataFrame.from_dict(pred_dict,orient='index')
sub.index.names = ['id']
sub.columns = ['rle_mask']
sub.to_csv('self_train.csv')

In [ ]:
n = 5
for i,fn in enumerate(train_ids[:n]):
    plt.subplot(1,n*2,i+1)
    plt.title(str(i))
    plt.imshow(np.dstack((Y_train[i],Y_train[i],Y_train[i])).astype(np.float32))

    
plt.show()
for i,fn in enumerate(train_ids[:n]):
    raw = cv2.imread("./data/train/masks/"+fn)
    plt.subplot(1,n*2,i+2)
    plt.title(str(i))
    plt.imshow(raw)
plt.show()

In [ ]:
print(train_ids[2730])

In [ ]:
#plt.imshow(np.squeeze(Y_train[2730],axis=-1))
result = cv2.resize(np.dstack((Y_train[2730],Y_train[2730],Y_train[2730])).astype(np.float32),(101,101))
plt.imshow(result)
print(result.shape)
plt.show()
raw_msk = cv2.imread("0aab0afa9c.png")
plt.figure()
plt.imshow(raw_msk)
plt.show()
plt.figure()
plt.imshow(np.dstack((X_train[2730],X_train[2730],X_train[2730])))
plt.show()